In [5]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings
import staintools

# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")

In [13]:
def svsToTiff(imageCount):
    global svsWidth,svsHeight,path,scene,fileName
    slide_tile_size=512
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)>0
            Full=np.mean(points)>=0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(256,256))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)/4:
                    TiffImageSave(image, fileName, widthCount,heightCount)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,widthCount,heightCount):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img,label
    img=Image.fromarray(image1) 
    img.save('../../data/tile_data/'+label+'/'+fileName+'/'+fileName+'_'+str(widthCount*512)+'_'+str(heightCount*512)+'.jpg')
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
      
    

In [11]:
normal_svs_files=glob('../../data/WHI/normal/*.tiff')
tumor_svs_files=glob('../../data/WHI/tumor/*.tiff')
normal_json_files = [f.replace('/WHI', '/json') for f in normal_svs_files]
normal_json_files = [f.replace('.tiff', '.json') for f in normal_json_files ]
tumor_json_files = [f.replace('/WHI', '/json') for f in tumor_svs_files]
tumor_json_files = [f.replace('.tiff', '.json') for f in tumor_json_files ]


In [15]:
for i in tqdm(range(len(tumor_svs_files))):
    file_extension_index=tumor_svs_files[i].find('.tiff')
    fileName=os.path.basename(os.path.splitext(tumor_svs_files[i])[0])
    slide = slideio.open_slide(tumor_svs_files[i], "GDAL")
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    imageCount=0
    magnification=80
    createDirectory('../../data/tile_data/tumor/'+fileName)
    tile_size=256
    label='tumor'
    with open(tumor_json_files[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    for j in range(polygon_count):
        polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])
        polygon1=np.copy(polygon)
        polygon1[:,0]=polygon[:,1]
        polygon1[:,1]=polygon[:,0]
        path = mpltPath.Path(polygon1)
        imageCount=svsToTiff(imageCount)
   


  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
for i in tqdm(range(len(normal_svs_files))):
    file_extension_index=normal_svs_files[i].find('.tiff')
    fileName=os.path.basename(os.path.splitext(normal_svs_files[i])[0])
    slide = slideio.open_slide(normal_svs_files[i], "GDAL")
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    imageCount=0
    magnification=80
    createDirectory('../../data/tile_data/normal/'+fileName)
    tile_size=256
    label='normal'
    with open(normal_json_files[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    for j in range(polygon_count):
        polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])
        polygon1=np.copy(polygon)
        polygon1[:,0]=polygon[:,1]
        polygon1[:,1]=polygon[:,0]
        path = mpltPath.Path(polygon1)
        imageCount=svsToTiff(imageCount)
   

  0%|          | 0/50 [00:00<?, ?it/s]